## Package

In [10]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from huggingface_hub import login
import json
import os
import ollama
import time
from datetime import datetime
from tqdm import tqdm
import re

## Get Data

In [ ]:
# 資料讀取選項
# 您可以選擇以下任一種方式來載入資料：

# 選項 1: 從已儲存的本地檔案讀取 (推薦，速度快)
use_local_files = True

# 選項 2: 從 Hugging Face 直接串流載入 (需要網路連線)
use_streaming = False

# 選項 3: 下載完整資料集 (檔案很大，不推薦)
use_full_download = False

print("=== 資料載入選項 ===")
print(f"使用本地檔案: {use_local_files}")
print(f"使用串流模式: {use_streaming}")
print(f"下載完整資料集: {use_full_download}")

# 資料載入
if use_local_files:
    print("\n📁 從本地檔案讀取資料...")
    
    # 檢查已儲存的檔案
    save_dir = "saved_datasets"
    
    if os.path.exists(save_dir):
        import glob
        
        # 尋找可用的檔案
        csv_files = glob.glob(f"{save_dir}/*.csv")
        json_files = glob.glob(f"{save_dir}/*.json")
        parquet_files = glob.glob(f"{save_dir}/*.parquet")
        
        print(f"找到的檔案:")
        print(f"  CSV 檔案: {len(csv_files)} 個")
        print(f"  JSON 檔案: {len(json_files)} 個")
        print(f"  Parquet 檔案: {len(parquet_files)} 個")
        
        # 優先使用 Parquet 檔案 (最高效)
        if parquet_files:
            latest_file = max(parquet_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 Parquet 檔案: {latest_file}")
            df = pd.read_parquet(latest_file)
            
        # 其次使用 CSV 檔案
        elif csv_files:
            latest_file = max(csv_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 CSV 檔案: {latest_file}")
            df = pd.read_csv(latest_file)
            
        # 最後使用 JSON 檔案
        elif json_files:
            latest_file = max(json_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 JSON 檔案: {latest_file}")
            with open(latest_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            df = pd.DataFrame(data)
            
        else:
            print("❌ 沒有找到已儲存的資料檔案")
            print("請先執行資料下載和儲存的程式碼")
            df = None
    else:
        print("❌ 找不到 saved_datasets 目錄")
        print("請先執行資料下載和儲存的程式碼")
        df = None

elif use_streaming:
    print("\n🌐 從 Hugging Face 串流載入資料...")
    
    # 使用串流模式載入資料集
    dataset = load_dataset("austenjs/ClueCorpusSmallDataset", streaming=True)
    
    # 設定要載入的樣本數量
    num_samples = 1000
    print(f"載入前 {num_samples} 筆資料...")
    
    # 收集資料
    sample_data = []
    for i, example in enumerate(dataset['train']):
        if i >= num_samples:
            break
        sample_data.append(example)
        if (i + 1) % 100 == 0:
            print(f"  已載入 {i + 1} 筆資料...")
    
    # 轉換為 DataFrame
    df = pd.DataFrame(sample_data)
    
elif use_full_download:
    print("\n⬇️ 下載完整資料集...")
    print("警告：這將下載 13.7GB 的資料，可能需要很長時間")
    
    # 下載完整資料集
    dataset = load_dataset("austenjs/ClueCorpusSmallDataset")
    df = dataset['train'].to_pandas()

else:
    print("❌ 沒有選擇任何資料載入選項")
    df = None

# 顯示資料資訊
if df is not None:
    print(f"\n✅ 資料載入成功！")
    print(f"📊 資料形狀: {df.shape}")
    print(f"📋 欄位名稱: {list(df.columns)}")
    
    # 顯示基本統計
    if 'text' in df.columns: # type: ignore
        df['text_length'] = df['text'].str.len() # type: ignore
        print(f"\n📈 文本長度統計:")
        print(df['text_length'].describe()) # type: ignore
        
        # 顯示前幾筆資料範例
        print(f"\n📝 前 3 筆資料範例:")
        for i in range(min(3, len(df))): # type: ignore
            text = df.iloc[i]['text']
            # 顯示前100個字符
            preview = text[:100] + "..." if len(text) > 100 else text
            print(f"範例 {i+1} ({len(text)} 字符): {preview}")
            print("-" * 80)
    
    print(f"\n🎯 資料已準備就緒，可用於後續的 LLM 評分處理！")
else:
    print("\n❌ 資料載入失敗，請檢查設定並重新執行")

# 儲存到全域變數供後續使用
globals()['dataset_df'] = df

=== 資料載入選項 ===
使用本地檔案: True
使用串流模式: False
下載完整資料集: False

📁 從本地檔案讀取資料...
找到的檔案:
  CSV 檔案: 1 個
  JSON 檔案: 1 個
  Parquet 檔案: 1 個

📊 讀取最新的 Parquet 檔案: saved_datasets/clue_corpus_small_20250901_085816.parquet

✅ 資料載入成功！
📊 資料形狀: (1000, 1)
📋 欄位名稱: ['text']

📈 文本長度統計:
count     1000.000000
mean       300.899000
std        785.763282
min          5.000000
25%         38.000000
50%        105.500000
75%        274.000000
max      17020.000000
Name: text_length, dtype: float64

📝 前 3 筆資料範例:
範例 1 (132 字符): 130真是佩服这家店开这么久。尽管门面已经小了一圈，但还是开着不容易啊。我们不容易，老板也不容易。自助餐，你可以吃得比平时多，但决不能浪费。想吃回20元，那是不可能的，所以还是不要去了。菜真的很一般，...
--------------------------------------------------------------------------------
範例 2 (8 字符): 送货速度奇慢无比
--------------------------------------------------------------------------------
範例 3 (12 字符): 这是自己用过最好的用的了
--------------------------------------------------------------------------------

🎯 資料已準備就緒，可用於後續的 LLM 評分處理！


## LLM AUG

In [11]:
# 使用 Ollama GPT-OSS 20B 進行資料擴增

print("🚀 開始使用 Ollama GPT-OSS 20B 進行資料擴增...")

# 檢查是否有載入的資料集
if 'dataset_df' not in globals() or dataset_df is None:
    print("❌ 沒有找到資料集，請先執行 GET DATA 部分")
else:
    # 設定參數
    model_name = "gpt-oss:20b"  # Ollama 模型名稱
    num_augmentations_per_text = 3  # 每個原始文本生成的擴增數量
    max_texts_to_process = 50  # 處理的文本數量 (可調整)
    
    print(f"📋 設定參數:")
    print(f"  模型: {model_name}")
    print(f"  每個文本擴增數量: {num_augmentations_per_text}")
    print(f"  處理文本數量: {max_texts_to_process}")
    
    # 檢查 Ollama 連接
    try:
        # 測試 Ollama 連接
        available_models = ollama.list()
        print(f"\n🔍 檢查 Ollama 模型...")
        model_found = any(model_name in model['name'] for model in available_models['models'])
        
        if not model_found:
            print(f"⚠️  模型 {model_name} 未找到，嘗試拉取模型...")
            print(f"正在下載 {model_name}，這可能需要一些時間...")
            ollama.pull(model_name)
            print(f"✅ 模型 {model_name} 下載完成")
        else:
            print(f"✅ 模型 {model_name} 已就緒")
            
    except Exception as e:
        print(f"❌ Ollama 連接錯誤: {e}")
        print("請確保 Ollama 服務正在運行")
        print("可以嘗試在終端機執行: ollama serve")
        
    # 定義資料擴增的提示模板
    augmentation_prompts = [
        # 提示1: 改寫保持原意
        """請將以下中文文本進行改寫，保持原意但使用不同的表達方式：

原文本：{text}

要求：
1. 保持文本的核心意思不變
2. 使用不同的詞彙和句式
3. 保持簡體中文
4. 輸出格式只需要改寫後的文本，不要其他說明

改寫文本：""",

        # 提示2: 風格轉換
        """請將以下文本轉換為更正式/更口語的表達方式：

原文本：{text}

要求：
1. 如果原文比較口語，請轉為正式表達
2. 如果原文比較正式，請轉為口語表達
3. 保持文本的主要資訊
4. 使用簡體中文
5. 輸出格式只需要轉換後的文本，不要其他說明

轉換文本：""",

        # 提示3: 句式重組
        """請重新組織以下文本的句子結構，但保持相同的含義：

原文本：{text}

要求：
1. 重新安排句子順序或結構
2. 可以將長句拆分或短句合併
3. 保持原始含義
4. 使用簡體中文
5. 輸出格式只需要重組後的文本，不要其他說明

重組文本："""
    ]
    
    # 資料擴增函數
    def augment_text(original_text, prompt_template):
        """使用 Ollama 擴增單個文本"""
        try:
            # 準備提示
            full_prompt = prompt_template.format(text=original_text)
            
            # 調用 Ollama
            response = ollama.generate(
                model=model_name,
                prompt=full_prompt,
                options={
                    'temperature': 0.7,
                    'top_p': 0.9,
                    'max_tokens': 1000
                }
            )
            
            # 提取生成的文本
            augmented_text = response['response'].strip()
            
            # 清理文本 (移除可能的前綴)
            augmented_text = re.sub(r'^(改寫文本：|轉換文本：|重組文本：)', '', augmented_text).strip()
            
            return augmented_text
            
        except Exception as e:
            print(f"⚠️ 文本擴增錯誤: {e}")
            return None
    
    # 開始資料擴增
    augmented_data = []
    original_data = []
    
    # 準備原始資料
    texts_to_process = dataset_df['text'].head(max_texts_to_process).tolist()
    
    print(f"\n🔄 開始處理 {len(texts_to_process)} 個文本...")
    
    # 使用進度條
    with tqdm(total=len(texts_to_process) * num_augmentations_per_text, desc="擴增進度") as pbar:
        
        for idx, original_text in enumerate(texts_to_process):
            # 添加原始文本
            original_data.append({
                'text': original_text,
                'source': 'original',
                'original_idx': idx
            })
            
            # 對每個文本進行多次擴增
            for aug_idx in range(num_augmentations_per_text):
                # 選擇提示模板 (輪流使用)
                prompt_idx = aug_idx % len(augmentation_prompts)
                prompt = augmentation_prompts[prompt_idx]
                
                # 進行擴增
                augmented_text = augment_text(original_text, prompt)
                
                if augmented_text and len(augmented_text.strip()) > 0:
                    augmented_data.append({
                        'text': augmented_text,
                        'source': f'augmented_v{aug_idx + 1}',
                        'original_idx': idx,
                        'augmentation_method': f'prompt_{prompt_idx + 1}'
                    })
                else:
                    print(f"⚠️ 第 {idx} 個文本的第 {aug_idx + 1} 次擴增失敗")
                
                pbar.update(1)
                
                # 添加小延遲避免過載
                time.sleep(0.1)
    
    # 合併原始資料和擴增資料
    all_data = original_data + augmented_data
    
    # 創建新的 DataFrame
    augmented_df = pd.DataFrame(all_data)
    
    print(f"\n✅ 資料擴增完成！")
    print(f"📊 擴增統計:")
    print(f"  原始文本: {len(original_data)} 筆")
    print(f"  擴增文本: {len(augmented_data)} 筆")
    print(f"  總計: {len(all_data)} 筆")
    print(f"  擴增倍率: {len(all_data) / len(original_data):.1f}x")
    
    # 顯示資料來源分布
    print(f"\n📈 資料來源分布:")
    print(augmented_df['source'].value_counts())
    
    # 顯示範例
    print(f"\n📝 擴增範例:")
    example_idx = 0
    original_example = augmented_df[augmented_df['original_idx'] == example_idx]
    
    for i, row in original_example.iterrows():
        print(f"\n{row['source'].upper()}:")
        text_preview = row['text'][:150] + "..." if len(row['text']) > 150 else row['text']
        print(f"  {text_preview}")
    
    # 儲存擴增後的資料集
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    augmented_dir = "augmented_datasets"
    os.makedirs(augmented_dir, exist_ok=True)
    
    # 儲存為多種格式
    base_filename = f"{augmented_dir}/clue_corpus_augmented_{timestamp}"
    
    # CSV 格式
    csv_filename = f"{base_filename}.csv"
    augmented_df.to_csv(csv_filename, index=False, encoding='utf-8')
    
    # JSON 格式
    json_filename = f"{base_filename}.json"
    augmented_df.to_json(json_filename, orient='records', ensure_ascii=False, indent=2)
    
    # Parquet 格式
    parquet_filename = f"{base_filename}.parquet"
    augmented_df.to_parquet(parquet_filename, index=False)
    
    print(f"\n💾 擴增資料集已儲存:")
    print(f"  CSV: {csv_filename}")
    print(f"  JSON: {json_filename}")
    print(f"  Parquet: {parquet_filename}")
    
    # 檔案大小
    for filename in [csv_filename, json_filename, parquet_filename]:
        size_mb = os.path.getsize(filename) / (1024 * 1024)
        print(f"  {os.path.basename(filename)}: {size_mb:.2f} MB")
    
    # 儲存到全域變數
    globals()['augmented_dataset_df'] = augmented_df
    
    print(f"\n🎯 擴增資料集已準備就緒，可用於訓練和評估！")
    print(f"變數名稱: augmented_dataset_df")

🚀 開始使用 Ollama GPT-OSS 20B 進行資料擴增...
📋 設定參數:
  模型: gpt-oss:20b
  每個文本擴增數量: 3
  處理文本數量: 50

🔍 檢查 Ollama 模型...
❌ Ollama 連接錯誤: 'name'
請確保 Ollama 服務正在運行
可以嘗試在終端機執行: ollama serve

🔄 開始處理 50 個文本...


擴增進度: 100%|██████████| 150/150 [36:07<00:00, 14.45s/it]


✅ 資料擴增完成！
📊 擴增統計:
  原始文本: 50 筆
  擴增文本: 150 筆
  總計: 200 筆
  擴增倍率: 4.0x

📈 資料來源分布:
source
original        50
augmented_v1    50
augmented_v2    50
augmented_v3    50
Name: count, dtype: int64

📝 擴增範例:

ORIGINAL:
  130真是佩服这家店开这么久。尽管门面已经小了一圈，但还是开着不容易啊。我们不容易，老板也不容易。自助餐，你可以吃得比平时多，但决不能浪费。想吃回20元，那是不可能的，所以还是不要去了。菜真的很一般，洗干净就好啦。什么都要另外付钱，一定要想好，别的不叫，只吃自助。

AUGMENTED_V1:
  真让人佩服这家店能经营这么久。虽然门面缩小了一圈，却依旧维持营业并不简单。我们和老板都不容易。自助餐可以吃得比平时多，但绝不能浪费。想要退回20元，根本不可能，还是别去吧。菜品真的很普通，干净就行。所有东西都要额外收费，务必三思，别点别的，只吃自助。

AUGMENTED_V2:
  我深感佩服这家店经营至今。尽管门面已略显简约，但仍不易维持。我们与老板同样面临诸多困难。自助餐可比平时多食，但务必避免浪费。若期望退回20元，恐怕无法实现，建议不再前往。菜品质量一般，只需洗净后食用。所有项目均需额外付费，请务必三思，若只想吃自助，请自行决定。

AUGMENTED_V3:
  这家店开了这么久，真让人佩服。门面虽已变小，但仍不易经营。我们和老板都不容易。自助餐可以比平时吃得多，但一定不能浪费。想要回20元是不可能的，还是别去了。菜品很一般，洗干净就行。所有东西都要另外付费，一定要想清楚，别叫别的菜，只吃自助。


TypeError: NDFrame.to_json() got an unexpected keyword argument 'ensure_ascii'

In [12]:
# 修正儲存問題並完成擴增資料集儲存
print("🔧 修正儲存格式並完成資料集儲存...")

if 'augmented_dataset_df' in globals():
    # 重新儲存
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    augmented_dir = "augmented_datasets"
    os.makedirs(augmented_dir, exist_ok=True)
    
    base_filename = f"{augmented_dir}/clue_corpus_augmented_{timestamp}"
    
    # CSV 格式
    csv_filename = f"{base_filename}.csv"
    augmented_dataset_df.to_csv(csv_filename, index=False, encoding='utf-8')
    
    # JSON 格式 (修正版本)
    json_filename = f"{base_filename}.json"
    with open(json_filename, 'w', encoding='utf-8') as f:
        json.dump(augmented_dataset_df.to_dict('records'), f, ensure_ascii=False, indent=2)
    
    # Parquet 格式
    parquet_filename = f"{base_filename}.parquet"
    augmented_dataset_df.to_parquet(parquet_filename, index=False)
    
    print(f"✅ 擴增資料集儲存完成:")
    print(f"  CSV: {csv_filename}")
    print(f"  JSON: {json_filename}")
    print(f"  Parquet: {parquet_filename}")
    
    # 檔案大小統計
    print(f"\n📁 檔案大小:")
    for filename in [csv_filename, json_filename, parquet_filename]:
        if os.path.exists(filename):
            size_mb = os.path.getsize(filename) / (1024 * 1024)
            print(f"  {os.path.basename(filename)}: {size_mb:.2f} MB")
    
    # 最終統計
    print(f"\n📊 最終擴增統計:")
    print(f"  資料集形狀: {augmented_dataset_df.shape}")
    print(f"  欄位: {list(augmented_dataset_df.columns)}")
    
    # 品質檢查
    print(f"\n🔍 資料品質檢查:")
    empty_texts = augmented_dataset_df['text'].str.strip().eq('').sum()
    duplicate_texts = augmented_dataset_df.duplicated(subset=['text']).sum()
    avg_length = augmented_dataset_df['text'].str.len().mean()
    
    print(f"  空白文本: {empty_texts} 筆")
    print(f"  重複文本: {duplicate_texts} 筆")
    print(f"  平均文本長度: {avg_length:.1f} 字符")
    
    # 擴增方法統計
    if 'augmentation_method' in augmented_dataset_df.columns:
        print(f"\n📈 擴增方法分布:")
        method_counts = augmented_dataset_df['augmentation_method'].value_counts()
        for method, count in method_counts.items():
            print(f"  {method}: {count} 筆")
    
    print(f"\n🎉 資料擴增專案完成！")
    print(f"🎯 原始資料集: 50 筆 → 擴增資料集: {len(augmented_dataset_df)} 筆")
    print(f"📈 擴增倍率: {len(augmented_dataset_df) / 50:.1f}x")
    
else:
    print("❌ 找不到擴增資料集變數，請重新執行擴增程序")

🔧 修正儲存格式並完成資料集儲存...
❌ 找不到擴增資料集變數，請重新執行擴增程序


In [13]:
# 檢查並讀取已存在的擴增資料 (如果有的話)
print("🔍 檢查擴增資料狀態...")

# 檢查是否有現有的擴增資料檔案
augmented_dir = "augmented_datasets"
if os.path.exists(augmented_dir):
    import glob
    
    # 尋找最新的擴增資料檔案
    parquet_files = glob.glob(f"{augmented_dir}/*.parquet")
    csv_files = glob.glob(f"{augmented_dir}/*.csv")
    
    if parquet_files:
        # 使用最新的 parquet 檔案
        latest_file = max(parquet_files, key=os.path.getctime)
        print(f"📂 找到擴增資料檔案: {latest_file}")
        augmented_dataset_df = pd.read_parquet(latest_file)
        print(f"✅ 成功載入擴增資料集")
        
    elif csv_files:
        # 使用最新的 CSV 檔案
        latest_file = max(csv_files, key=os.path.getctime)
        print(f"📂 找到擴增資料檔案: {latest_file}")
        augmented_dataset_df = pd.read_csv(latest_file)
        print(f"✅ 成功載入擴增資料集")
        
    else:
        print("❌ 沒有找到擴增資料檔案")
        augmented_dataset_df = None
else:
    print("❌ 擴增資料目錄不存在")
    augmented_dataset_df = None

# 如果成功載入，顯示統計資訊
if augmented_dataset_df is not None:
    print(f"\n📊 擴增資料集統計:")
    print(f"  資料形狀: {augmented_dataset_df.shape}")
    print(f"  欄位: {list(augmented_dataset_df.columns)}")
    
    # 資料來源分布
    if 'source' in augmented_dataset_df.columns:
        print(f"\n📈 資料來源分布:")
        print(augmented_dataset_df['source'].value_counts())
    
    # 文本長度統計
    augmented_dataset_df['text_length'] = augmented_dataset_df['text'].str.len()
    print(f"\n📏 文本長度統計:")
    print(augmented_dataset_df['text_length'].describe())
    
    # 顯示一些範例
    print(f"\n📝 擴增範例對比:")
    
    # 找一個原始文本和其擴增版本
    if 'original_idx' in augmented_dataset_df.columns:
        sample_idx = 0
        sample_data = augmented_dataset_df[augmented_dataset_df['original_idx'] == sample_idx]
        
        for i, row in sample_data.iterrows():
            source = row['source']
            text = row['text']
            preview = text[:100] + "..." if len(text) > 100 else text
            print(f"\n{source.upper()}:")
            print(f"  {preview}")
    
    # 品質檢查
    print(f"\n🔍 資料品質檢查:")
    empty_texts = augmented_dataset_df['text'].str.strip().eq('').sum()
    duplicate_texts = augmented_dataset_df.duplicated(subset=['text']).sum()
    
    print(f"  空白文本: {empty_texts} 筆")
    print(f"  重複文本: {duplicate_texts} 筆")
    
    # 計算擴增效果
    original_count = (augmented_dataset_df['source'] == 'original').sum()
    augmented_count = len(augmented_dataset_df) - original_count
    
    print(f"\n🎯 擴增效果總結:")
    print(f"  原始文本: {original_count} 筆")
    print(f"  擴增文本: {augmented_count} 筆")
    print(f"  總計: {len(augmented_dataset_df)} 筆")
    print(f"  擴增倍率: {len(augmented_dataset_df) / original_count:.1f}x")
    
    # 儲存到全域變數
    globals()['augmented_dataset_df'] = augmented_dataset_df
    
    print(f"\n✅ 擴增資料集已載入並準備就緒！")
    print(f"變數名稱: augmented_dataset_df")
    
else:
    print("\n❌ 無法載入擴增資料集")

🔍 檢查擴增資料狀態...
📂 找到擴增資料檔案: augmented_datasets/clue_corpus_augmented_20250901_094627.csv
✅ 成功載入擴增資料集

📊 擴增資料集統計:
  資料形狀: (200, 4)
  欄位: ['text', 'source', 'original_idx', 'augmentation_method']

📈 資料來源分布:
source
original        50
augmented_v1    50
augmented_v2    50
augmented_v3    50
Name: count, dtype: int64

📏 文本長度統計:
count     200.000000
mean      327.555000
std       423.366262
min         8.000000
25%        61.500000
50%       224.500000
75%       403.250000
max      2468.000000
Name: text_length, dtype: float64

📝 擴增範例對比:

ORIGINAL:
  130真是佩服这家店开这么久。尽管门面已经小了一圈，但还是开着不容易啊。我们不容易，老板也不容易。自助餐，你可以吃得比平时多，但决不能浪费。想吃回20元，那是不可能的，所以还是不要去了。菜真的很一般，...

AUGMENTED_V1:
  真让人佩服这家店能经营这么久。虽然门面缩小了一圈，却依旧维持营业并不简单。我们和老板都不容易。自助餐可以吃得比平时多，但绝不能浪费。想要退回20元，根本不可能，还是别去吧。菜品真的很普通，干净就行。所...

AUGMENTED_V2:
  我深感佩服这家店经营至今。尽管门面已略显简约，但仍不易维持。我们与老板同样面临诸多困难。自助餐可比平时多食，但务必避免浪费。若期望退回20元，恐怕无法实现，建议不再前往。菜品质量一般，只需洗净后食用。...

AUGMENTED_V3:
  这家店开了这么久，真让人佩服。门面虽已变小，但仍不易经营。我们和老板都不容易。自助餐可以比平时吃得多，但一定不能浪费。想要回20元是不可能的，还是别去了。菜品很一般，洗干净就行

## 🎯 最終版大陸用語識別與篩選系統

In [ ]:
# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "微信": [""], "支付寶": [""], "淘寶": [""]
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }

def mainland_score_ollama(text, model="gpt-oss:20b"):
    """使用 Ollama 評分大陸用語特徵"""
    prompt = f"""評估文本的大陸用語特徵，每項0或1分：

文本：{text}

評分標準：
1. 大陸特有詞彙：計算機、軟件、出租車、地鐵等
2. 大陸語法習慣：挺...的、蠻...的、咋樣等  
3. 大陸口語表達：搞定、整、弄等
4. 避免繁體用語：不含電腦、軟體、資料等
5. 整體大陸化程度：綜合評估

請按格式回答：
大陸特有詞彙:0
大陸語法習慣:0
大陸口語表達:0
避免繁體用語:1
整體大陸化程度:0
總分:1"""

    try:
        response = ollama.generate(
            model=model,
            prompt=prompt,
            options={'temperature': 0.1, 'max_tokens': 100}
        )
        
        # 解析回應
        scores = {}
        total = 0
        categories = ["大陸特有詞彙", "大陸語法習慣", "大陸口語表達", "避免繁體用語", "整體大陸化程度"]
        
        for line in response['response'].split('\n'):
            for cat in categories:
                if cat in line:
                    match = re.search(r'[：:]\s*([01])', line)
                    if match:
                        score = int(match.group(1))
                        scores[cat] = score
                        total += score
        
        if len(scores) == 5:
            scores["總分"] = total
            return scores, response['response']
        return None, response['response']
        
    except Exception as e:
        return None, str(e)

def process_dataset(df, text_col='text', sample_size=100, threshold=3):
    """處理資料集進行大陸用語篩選"""
    
    print(f"📊 處理資料集：{len(df)} 筆記錄")
    print(f"📝 文本欄位：{text_col}")
    print(f"🎯 樣本大小：{sample_size}")
    print(f"⚖️ 篩選閾值：{threshold}/5")
    
    # 取樣本
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    texts = sample_df[text_col].tolist()
    
    # 執行推論
    results = []
    authentic_texts = []
    generic_texts = []
    
    print(f"\n🔄 開始 Ollama 推論...")
    
    for i, text in enumerate(tqdm(texts, desc="推論進度")):
        # 特徵分析
        features = analyze_features(text)
        
        # Ollama 評分
        scores, response = mainland_score_ollama(text)
        
        result = {
            'index': i,
            'text': text,
            'text_length': len(text),
            'features': features,
            'scores': scores,
            'response': response,
            'success': scores is not None
        }
        
        # 分類
        if scores and scores.get("總分", 0) >= threshold:
            authentic_texts.append(result)
            category = "真正大陸用語"
        else:
            generic_texts.append(result)
            category = "通用簡體中文"
        
        result['category'] = category
        results.append(result)
        
        # 顯示進度
        if i % 20 == 0 or i < 3:
            score_str = f"{scores['總分']}/5" if scores else "失敗"
            print(f"  第{i+1}筆: {score_str} - {category}")
        
        time.sleep(0.2)  # 控制請求頻率
    
    return results, authentic_texts, generic_texts

def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據
    if authentic_texts:
        authentic_data = [{
            'text': r['text'],
            'total_score': r['scores']['總分'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        } for r in authentic_texts]
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集
available_data = None
text_column = 'text'

if 'augmented_dataset_df' in locals() and augmented_dataset_df is not None:
    available_data = augmented_dataset_df
    source_name = "擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    SAMPLE_SIZE = 50    # 處理樣本數量
    THRESHOLD = 3       # 篩選閾值
    
    print(f"\n🎯 開始執行大陸用語篩選...")
    results, authentic_results, generic_results = process_dataset(
        df=available_data,
        text_col=text_column,
        sample_size=SAMPLE_SIZE,
        threshold=THRESHOLD
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
✅ 使用 擴增資料集，共 200 筆記錄

🎯 開始執行大陸用語篩選...
📊 處理資料集：200 筆記錄
📝 文本欄位：text
🎯 樣本大小：50
⚖️ 篩選閾值：3/5

🔄 開始 Ollama 推論...


推論進度:   0%|          | 0/50 [00:00<?, ?it/s]

  第1筆: 1/5 - 通用簡體中文


推論進度:   2%|▏         | 1/50 [00:17<14:39, 17.94s/it]

  第2筆: 失敗 - 通用簡體中文


推論進度:   4%|▍         | 2/50 [07:34<3:31:27, 264.32s/it]

  第3筆: 1/5 - 通用簡體中文


推論進度:  40%|████      | 20/50 [14:44<13:54, 27.80s/it]  

  第21筆: 1/5 - 通用簡體中文


推論進度:  74%|███████▍  | 37/50 [34:07<03:49, 17.62s/it]   

## 🧪 30筆真實資料測試腳本

In [ ]:
# 🧪 30筆真實資料測試腳本 - 大陸用語識別系統驗證
print("🧪 啟動30筆真實資料測試...")

# 測試用的30筆真實資料 - 包含大陸用語和通用簡體中文
test_data = [
    # 明顯大陸用語 (1-10)
    "我的計算機軟件出了問題，需要重新安裝系統。",
    "打車去機場挺方便的，出租車很多。",
    "咋樣？這個質量還可以嗎？",
    "數據庫的硬件配置需要升級了。",
    "地鐵站裡的服務員態度蠻好的。",
    "用微信支付很方便，比現金快多了。",
    "晚飯想吃土豆燉牛肉，再來個西紅柿雞蛋湯。",
    "網絡連接有問題，程序運行不了。",
    "這事兒搞定了嗎？別忘了發信息給我。",
    "啥時候能到？公交車堵車了。",
    
    # 中等大陸特徵 (11-20)
    "今天天氣挺不錯的，適合出去走走。",
    "這個價格蠻合理的，性價比很高。",
    "辦公室的電腦需要安裝新軟件。",
    "淘寶上買東西很方便，快遞也快。",
    "地下停車場的車位挺緊張的。",
    "這個問題搞不定，需要請教專家。",
    "支付寶轉賬很安全，手續費也低。",
    "質量控制做得不錯，產品很穩定。",
    "數據分析的結果挺有意思的。",
    "網上購物越來越流行了。",
    
    # 通用簡體中文 (21-30) 
    "今天的會議討論了很多重要議題。",
    "學習新技術需要持續的努力和實踐。",
    "這本書的內容很豐富，值得仔細閱讀。",
    "運動對身體健康非常重要。",
    "時間管理是提高工作效率的關鍵。",
    "環境保護是我們共同的責任。",
    "科技發展改變了我們的生活方式。",
    "教育是社會進步的重要推動力。",
    "藝術創作需要靈感和技巧的結合。",
    "團隊合作能夠創造更大的價值。"
]

# 手動標註的標準答案 (用於評估準確性)
ground_truth = [
    # 明顯大陸用語 (1-10) - 應該得分 >= 3
    True, True, True, True, True, True, True, True, True, True,
    # 中等大陸特徵 (11-20) - 可能得分 2-3
    True, True, False, True, False, False, True, False, False, False,
    # 通用簡體中文 (21-30) - 應該得分 <= 2
    False, False, False, False, False, False, False, False, False, False
]

print(f"📊 測試資料統計:")
print(f"  總測試樣本: {len(test_data)} 筆")
print(f"  預期大陸用語: {sum(ground_truth)} 筆")
print(f"  預期通用中文: {len(ground_truth) - sum(ground_truth)} 筆")

# 重複使用現有的分析函數
def analyze_features(text):
    """快速特徵分析"""
    mainland_terms = {
        "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
        "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
        "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
        "土豆": ["馬鈴薯"], "西紅柿": ["番茄"], "搞定": ["完成"], "挺": ["很"],
        "咋": ["怎麼"], "啥": ["什麼"], "微信": [""], "支付寶": [""], "淘寶": [""]
    }
    
    mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]
    
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    found_terms = [term for term in mainland_terms if term in text]
    
    return {
        "mainland_terms": found_terms,
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }

def mainland_score_ollama_test(text, model="gpt-oss:20b"):
    """測試用的 Ollama 評分函數"""
    prompt = f"""評估以下中文文本的大陸用語特徵，按照5個標準各給0或1分：

文本：{text}

評分標準：
1. 大陸特有詞彙 (計算機、軟件、出租車、地鐵等): 0/1分
2. 大陸語法習慣 (挺...的、蠻...的、咋樣等): 0/1分  
3. 大陸口語表達 (搞定、整、弄等): 0/1分
4. 避免繁體用語 (不含電腦、軟體、資料等): 0/1分
5. 整體大陸化程度 (綜合判斷): 0/1分

請嚴格按照以下格式回答，每行一個分數：
大陸特有詞彙:0
大陸語法習慣:0
大陸口語表達:0
避免繁體用語:0
整體大陸化程度:0
總分:0"""

    try:
        response = ollama.generate(
            model=model,
            prompt=prompt,
            options={'temperature': 0.1, 'max_tokens': 150}
        )
        
        # 解析回應
        scores = {}
        categories = ["大陸特有詞彙", "大陸語法習慣", "大陸口語表達", "避免繁體用語", "整體大陸化程度"]
        
        response_text = response['response']
        
        for cat in categories:
            pattern = rf"{cat}[：:]\s*([01])"
            match = re.search(pattern, response_text)
            if match:
                scores[cat] = int(match.group(1))
        
        # 計算總分
        if len(scores) == 5:
            total = sum(scores.values())
            scores["總分"] = total
            return scores, response_text, True
        else:
            return None, response_text, False
            
    except Exception as e:
        return None, str(e), False

# 執行測試
print(f"\n🔄 開始執行測試...")
print("="*80)

test_results = []
correct_predictions = 0
total_processed = 0

# 測試每一筆資料
for i, text in enumerate(tqdm(test_data, desc="測試進度")):
    print(f"\n第 {i+1} 筆測試:")
    print(f"文本: {text[:50]}{'...' if len(text) > 50 else ''}")
    
    # 特徵分析
    features = analyze_features(text)
    print(f"特徵: {features['mainland_terms']} (模式:{features['pattern_matches']})")
    
    # Ollama 評分
    scores, response, success = mainland_score_ollama_test(text)
    
    if success and scores:
        total_score = scores["總分"]
        predicted_mainland = total_score >= 3  # 閾值為3
        actual_mainland = ground_truth[i]
        
        # 判斷預測是否正確
        is_correct = predicted_mainland == actual_mainland
        if is_correct:
            correct_predictions += 1
        
        print(f"評分: {total_score}/5 → {'大陸用語' if predicted_mainland else '通用中文'}")
        print(f"實際: {'大陸用語' if actual_mainland else '通用中文'} → {'✅' if is_correct else '❌'}")
        
        # 詳細分數
        score_details = " | ".join([f"{k}:{v}" for k, v in scores.items() if k != "總分"])
        print(f"詳細: {score_details}")
        
        total_processed += 1
        
    else:
        print(f"❌ 評分失敗: {response}")
        predicted_mainland = None
        is_correct = False
    
    # 儲存結果
    result = {
        'index': i,
        'text': text,
        'features': features,
        'scores': scores,
        'predicted_mainland': predicted_mainland,
        'actual_mainland': ground_truth[i],
        'correct': is_correct,
        'success': success
    }
    test_results.append(result)
    
    time.sleep(0.2)  # 控制請求頻率

# 計算測試結果統計
print("\n" + "="*80)
print("📊 測試結果統計:")

if total_processed > 0:
    accuracy = correct_predictions / total_processed * 100
    print(f"✅ 總體準確率: {accuracy:.1f}% ({correct_predictions}/{total_processed})")
    
    # 分類統計
    true_positives = sum(1 for r in test_results if r['predicted_mainland'] and r['actual_mainland'] and r['success'])
    true_negatives = sum(1 for r in test_results if not r['predicted_mainland'] and not r['actual_mainland'] and r['success'])
    false_positives = sum(1 for r in test_results if r['predicted_mainland'] and not r['actual_mainland'] and r['success'])
    false_negatives = sum(1 for r in test_results if not r['predicted_mainland'] and r['actual_mainland'] and r['success'])
    
    print(f"\n📈 分類詳細統計:")
    print(f"  真陽性 (正確識別大陸用語): {true_positives}")
    print(f"  真陰性 (正確識別通用中文): {true_negatives}")
    print(f"  假陽性 (誤判為大陸用語): {false_positives}")
    print(f"  假陰性 (遺漏大陸用語): {false_negatives}")
    
    # 計算精確率和召回率
    if (true_positives + false_positives) > 0:
        precision = true_positives / (true_positives + false_positives) * 100
        print(f"  精確率: {precision:.1f}%")
    
    if (true_positives + false_negatives) > 0:
        recall = true_positives / (true_positives + false_negatives) * 100
        print(f"  召回率: {recall:.1f}%")

# 分析錯誤案例
print(f"\n🔍 錯誤案例分析:")
errors = [r for r in test_results if not r['correct'] and r['success']]

if errors:
    print(f"發現 {len(errors)} 個錯誤案例:")
    for err in errors[:5]:  # 只顯示前5個錯誤
        text_preview = err['text'][:40] + "..." if len(err['text']) > 40 else err['text']
        predicted = "大陸" if err['predicted_mainland'] else "通用"
        actual = "大陸" if err['actual_mainland'] else "通用"
        score = err['scores']['總分'] if err['scores'] else "N/A"
        print(f"  ❌ 預測:{predicted} | 實際:{actual} | 分數:{score} | {text_preview}")
else:
    print("🎉 沒有發現錯誤案例！")

# 儲存測試結果
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
test_results_file = f"test_results_30samples_{timestamp}.csv"

# 準備儲存的資料
save_data = []
for r in test_results:
    row = {
        'text': r['text'],
        'predicted_mainland': r['predicted_mainland'],
        'actual_mainland': r['actual_mainland'],
        'correct': r['correct'],
        'authenticity_score': r['features']['authenticity_score'],
        'mainland_terms': ','.join(r['features']['mainland_terms'])
    }
    
    if r['scores']:
        row.update({f'score_{k.replace(":", "_")}': v for k, v in r['scores'].items()})
    
    save_data.append(row)

# 儲存為CSV
test_df = pd.DataFrame(save_data)
test_df.to_csv(test_results_file, index=False, encoding='utf-8-sig')

print(f"\n💾 測試結果已儲存: {test_results_file}")
print(f"📁 檔案大小: {os.path.getsize(test_results_file)/1024:.1f} KB")

# 儲存到全域變數
globals()['test_results_30'] = test_results
globals()['test_dataframe_30'] = test_df

print(f"\n🎯 30筆真實資料測試完成！")
print(f"📋 變數: test_results_30, test_dataframe_30")
print("="*80)